In [1]:
from MetaLearning.ApplicationPhase import ApplicationPhase
from MetaLearning import MetaFeatureExtractor
from pathlib import Path
from pandas.core.common import SettingWithCopyWarning
import warnings
warnings.filterwarnings(category=RuntimeWarning, action="ignore")
warnings.filterwarnings(category=SettingWithCopyWarning, action="ignore")
import numpy as np
np.random.seed(0)
# Specify where to find our MKR
# TODO: How to fix the path issue?
# mkr_path = Path("/home/tschecds/automlclustering/src/MetaKnowledgeRepository/")
mkr_path = Path("/home/camilo/dev/ml2dac/src/MetaKnowledgeRepository")

# Specify meta-feature set to use. This is the set General+Stats+Info 
mf_set = MetaFeatureExtractor.meta_feature_sets[4]
# Create simple synthetic dataset
from sklearn.datasets import make_blobs
# We expect the data as numpy arrays
X,y = make_blobs(n_samples=1000, n_features=10, random_state=0)

# We also use a name to describe/identify this dataset
dataset_name = "simple_blobs_n1000_f10"
# Parameters of our approach. This can be customized
n_warmstarts = 5 # Number of warmstart configurations (has to be smaller than n_loops)
n_loops = 10 # Number of optimizer loops. This is n_loops = n_warmstarts + x
limit_cs = True # Reduces the search space to suitable algorithms, dependening on warmstart configurations
time_limit = 120 * 60 # Time limit of overall optimization --> Aborts earlier if n_loops not finished but time_limit reached
cvi = "predict" # We want to predict a cvi based on our meta-knowledge
ML2DAC = ApplicationPhase(mkr_path=mkr_path, mf_set=mf_set)
optimizer_result, additional_info = ML2DAC.optimize_with_meta_learning(X, n_warmstarts=n_warmstarts,
                                                                       n_optimizer_loops=n_loops, 
                                                                       limit_cs=limit_cs,
                                                                       cvi=cvi, time_limit=time_limit,
                                                                       dataset_name=dataset_name)
best_config_stats = optimizer_result.get_incumbent_stats()
from sklearn.metrics import adjusted_rand_score
predicted_labels = best_config_stats["labels"]
adjusted_rand_score(predicted_labels, y)

In [10]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg')
from sklearn.preprocessing import LabelEncoder

def plot_pca_comparison(X, y, labels, ari_score, save_path):
    """Generate and save PCA plots comparing original labels and cluster labels."""
    def encode_labels(data):
        """Encode labels only if they are non-numeric."""
        if np.issubdtype(data.dtype, np.number):  # Check if numeric
            return data  # No changes for numeric data
        encoder = LabelEncoder()
        return encoder.fit_transform(data)  # Encode non-numeric labels

    # Encode y and labels dynamically based on their type
    y_encoded = encode_labels(np.array(y))
    labels_encoded = encode_labels(np.array(labels))

    # Perform PCA
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X)

    # Plot
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))

    scatter1 = axes[0].scatter(X_pca[:, 0], X_pca[:, 1], c=y_encoded, cmap='viridis', edgecolor='k')
    axes[0].set_title('PCA with Original Labels')
    axes[0].legend(*scatter1.legend_elements(), title="Classes")

    scatter2 = axes[1].scatter(X_pca[:, 0], X_pca[:, 1], c=labels_encoded, cmap='plasma', edgecolor='k')
    axes[1].set_title(f'PCA with Cluster Labels\nARI: {ari_score:.2f}')
    axes[1].legend(*scatter2.legend_elements(), title="Clusters")

    plt.tight_layout()
    plt.savefig(save_path, dpi=300)
    plt.close(fig)

In [11]:
from MetaLearning.ApplicationPhase import ApplicationPhase
from MetaLearning import MetaFeatureExtractor
from pathlib import Path
from pandas.core.common import SettingWithCopyWarning
import warnings
import pandas as pd
import numpy as np
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score, adjusted_rand_score
import os
import time

# Suppress warnings
warnings.filterwarnings(category=RuntimeWarning, action="ignore")
warnings.filterwarnings(category=SettingWithCopyWarning, action="ignore")
np.random.seed(0)
# Configuration
scoring_metric = "ml2dac"
validation_folder = "data/validation_csv"
results_file = f"results/{scoring_metric}/experiment_summary.csv"
output_folder = f"results/{scoring_metric}"
labels_folder = f"results/{scoring_metric}/labels"
os.makedirs(output_folder, exist_ok=True)
os.makedirs(labels_folder, exist_ok=True)
# Specify where to find the MetaKnowledgeRepository
mkr_path = Path("/home/camilo/dev/ml2dac/src/MetaKnowledgeRepository")

# Specify the meta-feature set to use (General+Stats+Info)
mf_set = MetaFeatureExtractor.meta_feature_sets[4]

# Initialize the ApplicationPhase instance
ML2DAC = ApplicationPhase(mkr_path=mkr_path, mf_set=mf_set)

# Parameters for the optimization process
n_warmstarts = 5
n_loops = 10
limit_cs = True
time_limit = 120 * 60  # 120 minutes time limit
cvi = "predict"  # Predict the CVI based on meta-knowledge

def get_processed_datasets(results_file):
    """Load processed datasets from results file."""
    if os.path.exists(results_file):
        df = pd.read_csv(results_file)
        return set(df['Dataset'])
    return set()

# Initialize the results file if it doesn't exist
if not os.path.exists(results_file):
    pd.DataFrame(columns=["Dataset", "Best_Pipeline", "silhouette_score", 
                          "davies_bouldin_score", "calinski_harabasz_score", 
                          "adjusted_rand_score", "Running_Time(s)"]
                ).to_csv(results_file, index=False)

processed_datasets = get_processed_datasets(results_file)

# Process each dataset in the directory
for dataset_name in os.listdir(validation_folder):
    try:
        start_time = time.time()

        # Load the dataset
        df = pd.read_csv(f"{validation_folder}/{dataset_name}")
        
        # Separate features and target (assuming the target is the last column)
        X = df.iloc[:, :-1].values
        y = df.iloc[:, -1].values
        print(f"\n{dataset_name}\n")
        # Perform the clustering optimization using ML2DAC
        optimizer_result, additional_info = ML2DAC.optimize_with_meta_learning(
            X, n_warmstarts=n_warmstarts, n_optimizer_loops=n_loops, 
            limit_cs=limit_cs, cvi=cvi, time_limit=time_limit,
            dataset_name=dataset_name
        )
        
        # Calculate the ARI using the predicted labels
        best_config_stats = optimizer_result.get_incumbent_stats()
        predicted_labels = best_config_stats["labels"]
        pipeline = best_config_stats['config']
        labels = np.array(predicted_labels)
        if(len(set(labels))>1):
            # Save labels to CSV for comparison
            labels_df = pd.DataFrame(labels, columns=["Cluster_Label"])
            labels_df.to_csv(f"{labels_folder}/{dataset_name.replace('.csv', '_labels.csv')}", index=False)
            
            sil = silhouette_score(X, labels)
            dbs = davies_bouldin_score(X, labels)
            chs = calinski_harabasz_score(X, labels)
        else:
            sil = None
            dbs = None
            chs = None
        ari = adjusted_rand_score(y, labels)
        running_time = round(time.time() - start_time, 2)
        # Save results
        results = pd.DataFrame({
            "Dataset": [dataset_name],
            "Best_Pipeline": [pipeline],
            "silhouette_score": [sil],
            "davies_bouldin_score": [dbs],
            "calinski_harabasz_score": [chs],
            "adjusted_rand_score": [ari],
            "Running_Time(s)": [running_time]
        })

        results.to_csv(results_file, mode="a", header=False, index=False)

        # Save PCA plot
        plot_pca_comparison(X, y, labels, ari, f"{output_folder}/{dataset_name.replace('.csv', '_pca.png')}")

    except Exception as e:
        print(f"Error processing {dataset_name}: {e}")


cluto-t7-10k.csv

----------------------------------
most similar dataset is: ['type=moons-k=2-n=10000-d=2-noise=0.1']
--
selected cvi: Silhouette (SIL)
--
Selected Warmstart Configs:
19    {'algorithm': 'dbscan', 'eps': 0.2948792654336...
24    {'algorithm': 'dbscan', 'eps': 0.2937667056363...
16    {'algorithm': 'dbscan', 'eps': 0.3144172336672...
21    {'algorithm': 'dbscan', 'eps': 0.2990119756196...
47    {'algorithm': 'dbscan', 'eps': 0.2983573713374...
Name: config, dtype: object
--
selected algorithms: ['dbscan']
--
----------------------------------
starting the optimization
Configuration space object:
  Hyperparameters:
    algorithm, Type: Categorical, Choices: {dbscan}, Default: dbscan
    eps, Type: UniformFloat, Range: [0.1, 1.0], Default: 0.55, Q: 0.01
    min_samples, Type: UniformInteger, Range: [2, 200], Default: 101
    n_clusters, Type: UniformInteger, Range: [2, 200], Default: 2
  Conditions:
    eps | algorithm in {'dbscan'}
    min_samples | algorithm in {'dbsca

/home/camilo/miniconda3/envs/ml2dac/lib/python3.9/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/home/camilo/miniconda3/envs/ml2dac/lib/python3.9/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.0.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


Obtained CVI score for SIL: -0.28289667337102564
----
Executing Configuration: Configuration(values={
  'algorithm': 'dbscan',
  'eps': 0.29376670563637275,
  'min_samples': 189,
})

Obtained CVI score for SIL: -0.2821028198281443
----
Executing Configuration: Configuration(values={
  'algorithm': 'dbscan',
  'eps': 0.31441723366726726,
  'min_samples': 190,
})

Obtained CVI score for SIL: -0.2984478853241852
----
Executing Configuration: Configuration(values={
  'algorithm': 'dbscan',
  'eps': 0.55,
  'min_samples': 101,
})

Obtained CVI score for SIL: 2147483647
----
Executing Configuration: Configuration(values={
  'algorithm': 'dbscan',
  'eps': 0.2990119756196803,
  'min_samples': 189,
})

Obtained CVI score for SIL: -0.2824922968766904
----
Executing Configuration: Configuration(values={
  'algorithm': 'dbscan',
  'eps': 0.29835737133741924,
  'min_samples': 196,
})

Obtained CVI score for SIL: -0.28329567401299643
----
Executing Configuration: Configuration(values={
  'algorithm

/home/camilo/miniconda3/envs/ml2dac/lib/python3.9/site-packages/pandas/core/frame.py:1485: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(



segment.csv

----------------------------------


...

/home/camilo/miniconda3/envs/ml2dac/lib/python3.9/site-packages/pymfe/_internal.py:1566: UserWarning: It is not possible make equal discretization
  warnings.warn("It is not possible make equal discretization")


most similar dataset is: ['type=gaussian-k=50-n=1000-d=30-noise=0']
--
selected cvi: DBCV (DBCV)
--
Selected Warmstart Configs:
30         {'algorithm': 'GMM', 'n_clusters': 51}
11         {'algorithm': 'GMM', 'n_clusters': 52}
10         {'algorithm': 'GMM', 'n_clusters': 53}
13         {'algorithm': 'GMM', 'n_clusters': 49}
34    {'algorithm': 'MBKMeans', 'n_clusters': 49}
Name: config, dtype: object
--
selected algorithms: ['GMM', 'MBKMeans']
--
----------------------------------
starting the optimization
Configuration space object:
  Hyperparameters:
    algorithm, Type: Categorical, Choices: {GMM, MBKMeans}, Default: GMM
    n_clusters, Type: UniformInteger, Range: [2, 200], Default: 2
  Conditions:
    n_clusters | algorithm in {'GMM', 'MBKMeans'}

Configuration space object:
  Hyperparameters:
    algorithm, Type: Categorical, Choices: {GMM, MBKMeans}, Default: GMM
    n_clusters, Type: UniformInteger, Range: [2, 200], Default: 2
  Conditions:
    n_clusters | algorithm in {'GMM

/home/camilo/miniconda3/envs/ml2dac/lib/python3.9/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/home/camilo/miniconda3/envs/ml2dac/lib/python3.9/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.0.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


Error occured: zero-size array to reduction operation maximum which has no identity
Obtained CVI score for DBCV: 1.0
----
Executing Configuration: Configuration(values={
  'algorithm': 'GMM',
  'n_clusters': 53,
})

Error occured: zero-size array to reduction operation maximum which has no identity
Obtained CVI score for DBCV: 1.0
----
Executing Configuration: Configuration(values={
  'algorithm': 'GMM',
  'n_clusters': 49,
})

Error occured: zero-size array to reduction operation maximum which has no identity
Obtained CVI score for DBCV: 1.0
----
Executing Configuration: Configuration(values={
  'algorithm': 'MBKMeans',
  'n_clusters': 49,
})

Error occured: zero-size array to reduction operation maximum which has no identity
Obtained CVI score for DBCV: 1.0
----
Executing Configuration: Configuration(values={
  'algorithm': 'GMM',
  'n_clusters': 124,
})

Error occured: zero-size array to reduction operation maximum which has no identity
Obtained CVI score for DBCV: 1.0
----
Executin

/home/camilo/miniconda3/envs/ml2dac/lib/python3.9/site-packages/pandas/core/frame.py:1485: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(
